In [1]:
import numpy as np
#import matplotlib.image as mpimg
#import re
#import cv2
#import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as data

In [2]:
train_transforms = transforms.Compose([transforms.Resize((224,224))])

In [3]:
train_data_images = torchvision.datasets.ImageFolder(root = 'training/images', loader = torchvision.io.read_image)
train_data_groundtruth = torchvision.datasets.ImageFolder(root = 'training/groundtruth', loader = torchvision.io.read_image)

In [4]:
print(train_data_groundtruth.size())

AttributeError: 'ImageFolder' object has no attribute 'size'

In [ ]:
# Red pixel value
print(train_data_images[0][0][0].size())
# Green pixel value
print(train_data_images[0][0][1].size())
# Blue pixel value
print(train_data_images[0][0][2].size())
#length train_data_images
print(len(train_data_images))

In [ ]:
train_images = []
for i in range (len(train_data_images)):
    train_images.append(torch.stack((train_data_images[i][0][0], train_data_images[i][0][1], train_data_images[i][0][2]), -1))

In [ ]:
train_groundtruth = []
for j in range (len(train_data_groundtruth)):
    train_groundtruth.append(train_data_groundtruth[i][0][0])

In [ ]:
print(train_images[0].size())

In [ ]:
train_loader_images = torch.utils.data.DataLoader(train_data_images,batch_size=4)
train_loader_groundtruth = torch.utils.data.DataLoader(train_data_groundtruth,batch_size=4)

In [ ]:
def set_device():
    if torch.cuda.is_available():
        dev ="cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [ ]:
class Model(nn.Module):
    # Models in PyTorch usually inherit from this Module
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 128)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
        '''
        self.input_layer = torch.nn.Linear(50176, 128)
        self.input_phi = torch.nn.ReLU()
        self.layer1 = torch.nn.Linear(128, 128)
        self.phi1 = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(128, 128)
        self.phi2 = torch.nn.ReLU()
        self.output_layer = torch.nn.Linear(128, 10)
        '''
    def forward(self, Z):
        # -> n, 3, 32, 32
        Z = self.pool(F.relu(self.conv1(Z)))  # -> n, 6, 14, 14
        Z = self.pool(F.relu(self.conv2(Z)))  # -> n, 16, 5, 5
        Z = Z.view(-1, 16 * 5 * 5)            # -> n, 400
        Z = F.relu(self.fc1(Z))               # -> n, 120
        Z = F.relu(self.fc2(Z))               # -> n, 84
        Z = self.fc3(Z)                       # -> n, 10
        
        '''
        Z = torch.flatten(Z, 1)  # Flatten (n, 28, 28) to (n, 784)
        Z = Z.to(torch.float32)
        Z = self.input_layer(Z)
        Z = self.input_phi(Z)

        Z = self.layer1(Z)
        Z = self.phi1(Z)

        Z = self.layer2(Z)
        Z = self.phi2(Z)

        Z = self.output_layer(Z)
        '''

        return Z       

In [ ]:
def train_epoch(model, device, train_data_images, train_data_groundtruth, optimizer, max_iter, criterion):
    model.train()
    
    loss_history = []
    accuracy_history = [] 
    
    #batch = np.random.choice(np.arange(len(train_data_images)), size=30, replace=False)
    
    
    for i, (images, labels) in enumerate(train_loader):        
        x = train_images[batch_idx]
        y = train_groundtruth[batch_idx]
        x = x.to(device)
        y = y.to(device)
        
        y_hat = model(x)  
                
        print(y_hat.size())
        print(y.size())
        
        loss = criterion(y_hat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = y_hat.argmax(dim=1, keepdim=True)
        correct = pred.eq(y.view_as(pred)).sum().item()

        loss_history.append(loss.item())
        accuracy_history.append(correct / len(x))

        if batch_idx % (len(train_data_images.dataset) // len(x) // 10) == 0:
            print(
                f"Train Epoch: {epoch}-{batch_idx} batch_loss={loss.item()/len(x):0.2e} batch_acc={correct/len(x):0.3f}"
            )

    return loss_history, accuracy_history
        
        

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
def run_mnist_training(num_epochs, lr, device):
    
    # ===== Model, Optimizer and Criterion =====
    model = Model()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.functional.cross_entropy
    
    # ===== Train Model =====
    train_loss_history = []
    train_acc_history = []
    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_epoch(
            model, device, train_data_images, train_data_groundtruth, optimizer, epoch, criterion)
        train_loss_history.extend(train_loss)
        train_acc_history.extend(train_acc)
        
    # ===== Plot training curves =====
    n_train = len(train_acc_history)
    t_train = num_epochs * np.arange(n_train) / n_train
    t_val = np.arange(1, num_epochs + 1)
    plt.figure()
    plt.plot(t_train, train_acc_history, label="Train")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")

    plt.figure()
    plt.plot(t_train, train_loss_history, label="Train")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

In [ ]:
lr = 5e-3
num_epochs = 2
device = set_device()

run_mnist_training(num_epochs, lr, device)